In [2]:
%pip install tensorflow
%pip install keras


  Using cached tensorflow-2.13.0-cp39-cp39-macosx_10_15_x86_64.whl (216.2 MB)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:

import pandas as pd
import json


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import datasets
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.data import Dataset
from keras.utils import to_categorical



2023-08-17 17:37:18.864682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
size = 32

In [17]:
base_model = ResNet50(include_top=False, pooling='avg',
                      input_shape=(size, size, 3), weights='imagenet')
base_model.trainable = False



In [18]:
inputs = Input(shape=(size, size, 3))
x = tf.keras.layers.experimental.preprocessing.Resizing(size, size)(inputs)
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = Flatten()(x)								# Fully Connected에 온전하게 학습을 위해 펼쳐준다
outputs = Dense(2, activation='softmax')(x)  # Softmax 함수로 1개 분류하는 분류기
model_res = tf.keras.Model(inputs, outputs)


In [19]:
model_res.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.__operators__.getitem_1  (None, 32, 32, 3)         0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_1 (TFOpLamb  (None, 32, 32, 3)         0         
 da)                                                             
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 4098

In [20]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.backend import eval

IMAGE_WIDTH, IMAGE_HEIGHT = size, size

def load_image_path(image_path_tensor):
        return image_path_tensor.decode('utf-8')

def load_image(image_path):
        with tf.io.gfile.GFile(image_path, 'rb') as f:
            img_bytes = f.read()
        img = tf.image.decode_image(img_bytes, channels=3)
        img = tf.image.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
        img = tf.cast(img, tf.float32) / 255.0
        return img


def load_image_and_label(image_path_t, label): 
    tf.compat.v1.enable_eager_execution()
    image_path = tf.numpy_function(load_image_path, [image_path_t], tf.string)
    image = tf.numpy_function(load_image, [image_path], tf.float32)
    
    return image, label

In [21]:
# CSV 파일의 경로 설정
train_csv_file_path = "/Users/jaeiklee/Documents/GitHub/cbnu/projects/laundryStain/TensorFlowDataSet_custom_cat_doc/train/_annotations.csv"

# pandas를 사용하여 CSV 파일 로드
train_dataframe = pd.read_csv(train_csv_file_path)

# 입력 특성과 레이블로 데이터 분할

images = train_dataframe['filename'].tolist()
labels = train_dataframe['class'].tolist()

label_mapping = {label: index for index, label in enumerate(set(labels))}
encoded_labels = [label_mapping[label] for label in labels]
print(label_mapping)
encoded_labels = to_categorical(encoded_labels,2)

train_dataset = Dataset.from_tensor_slices(
    (images, encoded_labels))
train_dataset = train_dataset.map(load_image_and_label)


# (선택적) 데이터 셔플, 배치 크기 설정
batch_size = 100
buffer_size = 2000

train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)


{'dog': 0, 'cat': 1}


In [22]:
# CSV 파일의 경로 설정
valid_csv_file_path = "/Users/jaeiklee/Documents/GitHub/cbnu/projects/laundryStain/TensorFlowDataSet_custom_cat_doc/valid/_annotations.csv"

# pandas를 사용하여 CSV 파일 로드
valid_dataframe = pd.read_csv(valid_csv_file_path)

# 입력 특성과 레이블로 데이터 분할

images = valid_dataframe['filename'].tolist()
labels = valid_dataframe['class'].tolist()

label_mapping = {label: index for index, label in enumerate(set(labels))}
print(label_mapping)
encoded_labels = [label_mapping[label] for label in labels]

encoded_labels = to_categorical(encoded_labels, 2)
print(encoded_labels)
valid_dataset = Dataset.from_tensor_slices(
    (images, encoded_labels))
valid_dataset = valid_dataset.map(load_image_and_label)


# (선택적) 데이터 셔플, 배치 크기 설정
batch_size = 100
buffer_size = 2000

valid_dataset = valid_dataset.shuffle(buffer_size).batch(batch_size)


{'dog': 0, 'cat': 1}
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [23]:
# 모델 컴파일 진행 - 아까와 달리 categorical_crossentropy 사용 > label이 숫자형 데이터이므로
model_res.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# # early stopping 설정
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# 모델 fitting
# save = model_res.fit(train_dataset, epochs=50,
#               validation_data=valid_dataset, batch_size=64, callbacks=[early])
save = model_res.fit(train_dataset, epochs=500,
                     validation_data=valid_dataset, batch_size=100)


Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 0.7164 - accuracy: 0.6296 - val_loss: 0.6817 - val_accuracy: 0.5833
Epoch 2/500
1/1 [==============================] - 0s 221ms/step - loss: 0.6553 - accuracy: 0.6296 - val_loss: 0.6899 - val_accuracy: 0.7500
Epoch 3/500
1/1 [==============================] - 0s 256ms/step - loss: 0.6911 - accuracy: 0.4444 - val_loss: 0.6804 - val_accuracy: 0.5000
Epoch 4/500
1/1 [==============================] - 0s 195ms/step - loss: 0.6725 - accuracy: 0.6667 - val_loss: 0.6766 - val_accuracy: 0.5833
Epoch 5/500
1/1 [==============================] - 0s 207ms/step - loss: 0.6466 - accuracy: 0.6296 - val_loss: 0.6987 - val_accuracy: 0.5833
Epoch 6/500
1/1 [==============================] - 0s 203ms/step - loss: 0.6465 - accuracy: 0.6296 - val_loss: 0.7218 - val_accuracy: 0.5833
Epoch 7/500
1/1 [==============================] - 0s 203ms/step - loss: 0.6565 - accuracy: 0.6296 - val_loss: 0.7213 - val_accuracy: 0.5833
Epoch 8/500
1/1 

In [237]:
model_res.save('resnet50_s32_e50_tbt_tt.h5')


/Users/jaeiklee/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
import matplotlib.pyplot as plt
from keras.applications.resnet50 import decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np


In [14]:
# CSV 파일의 경로 설정
test_csv_file_path = "/Users/jaeiklee/Documents/GitHub/cbnu/projects/laundryStain/TensorFlowDataSet_custom_cat_doc/test/_annotations.csv"

# pandas를 사용하여 CSV 파일 로드
test_dataframe = pd.read_csv(test_csv_file_path)

# 입력 특성과 레이블로 데이터 분할

images = test_dataframe['filename'].tolist()
labels = test_dataframe['class'].tolist()

label_mapping = {label: index for index, label in enumerate(set(labels))}
print(label_mapping)
encoded_labels = [label_mapping[label] for label in labels]

encoded_labels = to_categorical(encoded_labels, 2)
print(encoded_labels[:20])
test_dataset = Dataset.from_tensor_slices(
    (images, encoded_labels))
test_dataset = test_dataset.map(load_image_and_label)

# (선택적) 데이터 셔플, 배치 크기 설정
batch_size = 32
buffer_size = 1000

test_dataset = test_dataset.shuffle(buffer_size).batch(batch_size)



{'dog': 0, 'cat': 1}
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [193]:
from PIL import Image

#{'ink': 0, 'kimchi': 1, 'coffee': 2}
# [1,0,0]    [0,1,0]       [0,0,1]

In [30]:
predictions = model_res.evaluate(test_dataset)
print(predictions)
#traning True


1/1 [==============================] - 0s 114ms/step - loss: 0.1952 - accuracy: 0.9091
[0.19516371190547943, 0.9090909361839294]


In [27]:
predictions = model_res.evaluate(test_dataset)
print(predictions)
#traning False

1/1 [==============================] - 0s 86ms/step - loss: 0.1952 - accuracy: 0.9091
[0.19516421854496002, 0.9090909361839294]
